In [5]:
%pip install langchain-core langgraph>0.2.27 langchain langchain-openai langchain-community python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "default"
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = init_chat_model("gpt-4o-mini", model_provider="openai")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

Query not requiring any tools

In [3]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in sf
================================== Ai Message ==================================

Hi Bob! It's great to meet you. How's life in San Francisco?


Query requiring tool use.

In [4]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_F9yEgchRYLUrBw5detOYoeMA)
 Call ID: call_F9yEgchRYLUrBw5detOYoeMA
  Args:
    query: San Francisco weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "March 2025 Weather History in San Francisco California, United ...", "url": "https://weatherspark.com/h/m/557/2025/3/2025%E5%B9%B43%E6%9C%88%E3%81%AE%E3%82%B5%E3%83%B3%E3%83%95%E3%83%A9%E3%83%B3%E3%82%B7%E3%82%B9%E3%82%B3%E3%80%81%E3%82%AB%E3%83%AA%E3%83%95%E3%82%A9%E3%83%AB%E3%83%8B%E3%82%A2%E5%B7%9E%E3%80%81%E3%82%A2%E3%83%A1%E3%83%AA%E3%82%AB%E5%90%88%E8%A1%86%E5%9B%BD%E3%81%AB%E3%81%8A%E3%81%91%E3%82%8B%E6%B0%97%E5%80%99%E5%B1%A5%E6%AD%B4", "content": "Latest Report — 5:56 PM ; Temp. 57.0°F ; Wind. 9.2 mph ; Cloud 